In [1]:
import os
import sys
from glob import glob
import numpy as np
import pandas as pd
#import cv2
from PIL import Image
from tqdm.notebook import tqdm
from time import time
import math

import matplotlib.pyplot as plt
import seaborn as sns
import multiprocessing as mp

from sklearn.metrics import f1_score

import torch
from torch import nn
import torch.nn.functional as F
import torch.utils.data as data
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision import transforms
from torchvision.transforms import Resize, ToTensor, Normalize, GaussianBlur, RandomRotation, ColorJitter
from efficientnet_pytorch import EfficientNet
from torch.utils.tensorboard import SummaryWriter # 학습 중 실시간으로 acc와 loss 같은 값들을 그래프에 찍어서 보여주는 역할
from torch.optim import lr_scheduler # validation dataset을 학습시킬 때 조정해주는 scheduler
#import dataset_hs

from torchvision.transforms import Resize, ToTensor, Normalize, Compose, CenterCrop, ColorJitter, RandomHorizontalFlip, RandomRotation, RandomPerspective, Grayscale, Pad


In [2]:
#torch.cuda.is_available()
torch.cuda.get_device_name(0)

'Tesla V100-PCIE-32GB'

In [3]:
# Set random seed
import random
SEED = 2021
random.seed(SEED)
np.random.seed(SEED)
os.environ["PYTHONHASHSEED"] = str(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)  # type: ignore
torch.backends.cudnn.deterministic = True  # type: ignore
torch.backends.cudnn.benchmark = True  # type: ignore

In [4]:
class MaskDataset(Dataset):
    def __init__(self, table, transform):
        self.table = table
        self.transform = transform

        self.X, self.y = self.read_data()
        
    def read_data(self):
        X,y = [],[]
        for i in tqdm(range(len(self.table))):
            img = Image.open(self.table['absolute_path'][i])
            if self.transform:
                img = self.transform(img)
            X.append(img)
            y.append(torch.tensor(self.table['label'][i]))
        return X,y

    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]
        

In [ ]:
val_transform = transforms.Compose([
    Resize((int(224), int(224)), Image.BILINEAR),
    ToTensor(),
    # Normalize(mean=(0.548, 0.504, 0.479), std=(0.237, 0.247, 0.246)),
    ])

In [5]:
t_label = pd.read_csv('/opt/ml/tmp/train_image_label.csv')
v_label = pd.read_csv('/opt/ml/tmp/val_image_label.csv')

In [6]:
# transform = transforms.Compose([
#         Resize((int(224), int(224))),
#         ToTensor(),
#         Normalize(mean=(0.548, 0.504, 0.479), std=(0.237, 0.247, 0.246)),
#     ])

transform = transforms.Compose([
    CenterCrop((int(423), int(292))),
    ColorJitter(brightness=0.5, contrast=0.5, hue=0.5),
    RandomHorizontalFlip(p=0.5),
    RandomRotation(10),
    RandomPerspective(p=0.25, distortion_scale=0.25 ),
    Resize((int(224), int(224)), Image.BILINEAR),
    ToTensor(),
    Normalize(mean=(0.548, 0.504, 0.479), std=(0.237, 0.247, 0.246)),
    ])

train_dataset = MaskDataset(
    table=t_label,
    transform=transform
)

valid_dataset = MaskDataset(
    table=v_label,
    transform=val_transform
)

/opt/conda/lib/python3.8/site-packages/torchvision/transforms/transforms.py:287: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  warnings.warn(


In [7]:

train_loader = data.DataLoader(
    train_dataset,
    batch_size=32,
    num_workers=4, 
#     cuda설정이안돼서 num_workers 설정하면 안돌아감
    shuffle=True
)

val_loader = data.DataLoader(
    valid_dataset,
    batch_size=32,
    num_workers=4,
    shuffle=False
)
loader={'train':train_loader,
       'val':val_loader}

In [8]:
images, labels = next(iter(train_loader))
print(f'images shape: {images.shape}')
print(f'labels shape: {labels.shape}')

images shape: torch.Size([32, 3, 224, 224])
labels shape: torch.Size([32])


In [9]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [10]:
from torchsummary import summary
model = EfficientNet.from_pretrained('efficientnet-b7', num_classes=18) # b0~b8호출 가능, 학습시키는 데이터에 대한 num_classes
                                                                        #gpu memory가 8g 아래면 b3을 추천
model.to(device)
summary(model, input_size=(3,256,192))

Loaded pretrained weights for efficientnet-b7
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
         ZeroPad2d-1          [-1, 3, 257, 193]               0
Conv2dStaticSamePadding-2          [-1, 64, 128, 96]           1,728
       BatchNorm2d-3          [-1, 64, 128, 96]             128
MemoryEfficientSwish-4          [-1, 64, 128, 96]               0
         ZeroPad2d-5          [-1, 64, 130, 98]               0
Conv2dStaticSamePadding-6          [-1, 64, 128, 96]             576
       BatchNorm2d-7          [-1, 64, 128, 96]             128
MemoryEfficientSwish-8          [-1, 64, 128, 96]               0
          Identity-9             [-1, 64, 1, 1]               0
Conv2dStaticSamePadding-10             [-1, 16, 1, 1]           1,040
MemoryEfficientSwish-11             [-1, 16, 1, 1]               0
         Identity-12             [-1, 16, 1, 1]               0
Conv2dStaticSamePadding-13        

In [11]:
print("입력 채널 개수", model._conv_stem.weight.shape[1])
print("출력 채널(예측 class type 개수)", model._fc.weight.shape[0])

입력 채널 개수 3
출력 채널(예측 class type 개수) 18


In [12]:
learning_rate = 0.0001
num_epoch = 9
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer,'min') 
# ReduceLROnPlateau: val_loss가 향상되지 않고 정체되어 있으면 learning rate를 factor배로 감소시킴(factor 디폴트값:0.1)
# 'min' mode에서는 val_loss가 감소를 멈줬을 때 감소, 'max' mode에서는 val_loss가 증가를 멈췄을 때 감소 (mode 디폴트값:'min')

In [13]:
best_val_accuracy = 0.
best_val_loss = 9999.

torch.cuda.empty_cache()

for epoch in range(num_epoch):
    for phase in ["train", "val"]:
        running_loss = 0.
        running_acc = 0.
        n_iter = 0
        epoch_f1 = 0
        if  phase == "val":
            confusion_matrix = np.zeros((18, 18))

        if phase == "train":
            model.train()
        elif phase == "val":
            model.eval()

        for ind, data in enumerate(tqdm(loader[phase])):
            try:
                images = data[0].to(device)
                labels = data[1].to(device)

                optimizer.zero_grad() # parameter gradient를 업데이트 전 초기화함

                with torch.set_grad_enabled(phase == "train"): # train 모드일 시에는 gradient를 계산
                    logits = model(images)
                    _, preds = torch.max(logits, 1)
                    loss = loss_fn(logits, labels)

                    if phase == "train":
                        loss.backward() 
                        optimizer.step()
                # Metrics 계산 부분 ==============================================================================
                epoch_f1 += f1_score(labels.cpu().numpy(), preds.cpu().numpy(), average='macro')
                n_iter += 1
                if  phase == "val":
                    for t, p in zip(labels.view(-1), preds.view(-1)): # confusion matrix에 값 입력, 언제가 최적일 지 몰라 매 epoch돌아감
                        confusion_matrix[t.long(), p.long()] += 1    
                running_loss += loss.item() * images.size(0) # 한 Batch에서의 loss 값, images.size(0) = batch size
                running_acc += torch.sum(preds == labels.data) # 한 Batch에서의 Accuracy 값
            except:
                continue
            
            
    # 한 epoch이 모두 종료
    epoch_loss = running_loss / len(loader[phase].dataset)
    epoch_acc = running_acc / len(loader[phase].dataset)
    epoch_f1 = epoch_f1/n_iter

    print(f"epoch-{epoch}의 {phase}-데이터 평균 Loss : {epoch_loss:.3f}, 평균 Accuracy : {epoch_acc:.3f}, 평균 f1:{epoch_f1}")
    if phase == "val" and best_val_accuracy < epoch_acc: 
        best_val_accuracy = epoch_acc
    if phase == "val" and best_val_loss > epoch_loss: 
        best_val_loss = epoch_loss
print("학습 종료!")
print(f"최고 accuracy : {best_val_accuracy:3f}, 최고 낮은 loss : {best_val_loss:3f}")


epoch-0의 val-데이터 평균 Loss : 0.296, 평균 Accuracy : 0.896, 평균 f1:0.8213277139288462



epoch-1의 val-데이터 평균 Loss : 0.346, 평균 Accuracy : 0.891, 평균 f1:0.7970813934848014



epoch-2의 val-데이터 평균 Loss : 0.364, 평균 Accuracy : 0.903, 평균 f1:0.803766459357486



epoch-3의 val-데이터 평균 Loss : 0.411, 평균 Accuracy : 0.898, 평균 f1:0.8173924053452928



epoch-4의 val-데이터 평균 Loss : 0.459, 평균 Accuracy : 0.902, 평균 f1:0.8249578469222582



epoch-5의 val-데이터 평균 Loss : 0.444, 평균 Accuracy : 0.899, 평균 f1:0.8217331083798449



epoch-6의 val-데이터 평균 Loss : 0.485, 평균 Accuracy : 0.895, 평균 f1:0.793459395466976



epoch-7의 val-데이터 평균 Loss : 0.594, 평균 Accuracy : 0.879, 평균 f1:0.776898050820034



epoch-8의 val-데이터 평균 Loss : 0.482, 평균 Accuracy : 0.897, 평균 f1:0.8100778099722625
학습 종료!
최고 accuracy : 0.903175, 최고 낮은 loss : 0.296088


In [14]:
!pip install datetime

In [16]:
# 파일을 저장
from pytz import timezone
import datetime as dt

now = (dt.datetime.now().astimezone(timezone("Asia/Seoul")).strftime("%Y-%m-%d-%H-%M"))
#now
torch.save(model,f"/opt/ml/tmp/save_model/{now}.pth")

In [17]:
now = (dt.datetime.now().astimezone(timezone("Asia/Seoul")).strftime("%Y-%m-%d-%H-%M"))
model = torch.load(f'/opt/ml/tmp/save_model/{now}.pth')

In [18]:
class TestDataset(Dataset):
    def __init__(self, img_paths, transform):
        self.img_paths = img_paths
        self.transform = transform

    def __getitem__(self, index):
        image = Image.open(self.img_paths[index])

        if self.transform:
            image = self.transform(image)
        return image

    def __len__(self):
        return len(self.img_paths)

In [20]:
test_dir = '/opt/ml/input/data/eval'
submission = pd.read_csv(os.path.join(test_dir, 'info.csv'))
image_dir = os.path.join(test_dir, 'images')
image_paths = [os.path.join(image_dir, img_id) for img_id in submission.ImageID]

transform = transforms.Compose([
            Resize((int(224), int(224))),
            ToTensor(),
            #Normalize(mean=mean, std=std),
        ])
dataset = TestDataset(image_paths, transform)
loader = DataLoader(
    dataset,
    shuffle=False
)

In [21]:
# 모델을 정의합니다. (학습한 모델이 있다면 torch.load로 모델을 불러주세요!)
device = torch.device('cuda')
model = model.to(device)
model.eval()

all_predictions = []
for images in loader:
    with torch.no_grad():
        images = images.to(device)
        pred = model(images)
        pred = pred.argmax(dim=-1)
        all_predictions.extend(pred.cpu().numpy())
submission['ans'] = all_predictions

In [23]:
submission.to_csv(f'/opt/ml/tmp/submission/b7customsubmission.csv', index=False)
print('test inference is done!')

test inference is done!
